In [ ]:
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# Check for CPU/GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_transform = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(20),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
train_data = datasets.ImageFolder(root="../input/waste-segregation-image-dataset/Dataset/train", transform=train_transform)
trainDataLoader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)


test_transform = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
test_data = datasets.ImageFolder(root="../input/waste-segregation-image-dataset/Dataset/val", transform=test_transform)
testDataLoader = DataLoader(dataset=test_data, batch_size=32, shuffle=True)

# Load Pre-trained model
model = models.resnet50(pretrained=True)

print("Before: \n", model, "\n\n")

# Freeze our Feature parameters of the model
# Model Parameters: Pre-Learned Weights & Biases
for param in model.parameters():
    # Don't allow gradients/features to update.
    param.requires_grad = False

# Replace the Classification layer with our custom classification layer
from collections import OrderedDict

# Define a sequence of custom layers
# Pass in a OrderedDict to name each of these layers and corresponding functions
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(2048, 512)),
    ('relu', nn.ReLU()),
    ('dropout', nn.Dropout(p=0.2)),
    ('fc2', nn.Linear(512, 2)),
    ('output', nn.LogSoftmax(dim=1))
]))

# Add custom layers to the pre-trained model
model.fc = classifier

print("After: \n", model, "\n")

# Train the model with newly added classifier layer
model.to(device)

# Epochs
epochs = 10

# Loss Criterion
criterion = nn.NLLLoss()

# Optimizer
# Since, we only need to update the parameters for the Model classifier.
# So, we use "model.classifier.parameters()" instead of "model.parameters()".
optimizer = optim.Adam(params=model.fc.parameters(), lr=0.001)

train_losses, test_losses = [], []

print("Starting Model Training...")

# ------------- Training Loop ------------
for e in range(epochs):
    running_loss = 0
    # ------------- Training Loop -----------
    for images, labels in trainDataLoader:
        images, labels = Variable(images), Variable(labels)
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model.forward(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # ------------ Validation Loop -----------
    else:
        test_loss = 0
        accuracy = 0
        with torch.no_grad():
            model.eval()
            for images, labels in testDataLoader:
                images, labels = Variable(images), Variable(labels)
                images, labels = images.to(device), labels.to(device)
                logps = model.forward(images)
                test_loss += criterion(logps, labels)
                # Actual Probability distribution from Log Probabilities
                ps = torch.exp(logps)
                # topk: returns K largest elements of the given input tensor along a given dimension
                top_k, top_class = ps.topk(1, dim=1)
                # Check if Predicted Output is equal to the Actual Labels
                equals = top_class == labels.view(*top_class.shape)
                # Calculate Accuracy on the Test Dataset
                accuracy += torch.mean(equals.type(torch.FloatTensor))

        # ** Trun back ON the Dropouts for Model Training **
        model.train()

        # Keep track of Training and Test Loss
        train_losses.append(running_loss / len(trainDataLoader))
        test_losses.append(test_loss / len(testDataLoader))

        print("Epoch: {}/{}".format(e + 1, epochs),
              "Training Loss: {:.3f}\t".format(running_loss / len(trainDataLoader)),
              "Test Loss: {:.3f}\t".format(test_loss / len(testDataLoader)),
              "Test Accuracy: {:.3f}\t".format(accuracy / len(testDataLoader)))

print("\nDone...")